# KIC 8462852 Century Long Data Analysis

## Context

KIC 8462852 is an enigmatic star (Boyajian et al. 2015).


## Content

- Data from the DASCH project: APASS input catalog, Kepler input catalog and GSC 2.3 catalog.
- Sonneberg and Sternberg observatory data provided by Michael Hippke.
- Maria Mitchell observatory data provided by Michael Castelaz.


## References

1. Boyajian, et al. (2015). Planet Hunters X. KIC 8462852 - Where's the Flux? arXiv:1509.03622
2. Castelaz and Barker (2018). KIC 8462852: Maria Mitchell Observatory Photographic Photometry 1922 to 1991. arXiv:1803.01943
3. Hippke et al. (2017). Sonneberg plate photometry for Boyajian's Star in two passbands. arXiv:1609.09290


In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it

In [2]:
import numpy as np # main library for numeric calculations
import pandas as pd # main library for data analysis
import matplotlib.pyplot as plt # main library for data plotting
import seaborn as sns # Another library for data plotting with more functions
sns.set()

import scipy as scp # STEM software
from scipy import signal
import scipy.interpolate as interpolate

from glob import glob # check files
from IPython.display import display, Markdown

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import lightkurve as lk
import astropy

import sys, os

file_path = os.getcwd()
print(file_path)
parent_dir = "\\".join(file_path.split("\\")[:-1])
print(parent_dir)

sys.path.append(parent_dir)

from kepler import *

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

import warnings
warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter


def printf(*args, sep=" ", end="\n"):
    string = sep.join([str(i) for i in args])+end
    display(Markdown(string))


def evaluateAndProcessData(x, y):
    t, c, k = interpolate.splrep(x, y, s=0, k=4)
    xx = np.linspace(x.min(), x.max(), 100)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    
    plt.figure(figsize=(16,6))
    plt.plot(x, y, 'bo', label='Original points')
    plt.plot(x, y)
    plt.plot(xx, spline(xx), 'r', label='BSpline')
    plt.grid()
    plt.legend(loc='best')
    plt.show()


c:\users\prannaya\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


C:\Users\Prannaya\ThreeBody\notebooks
C:\Users\Prannaya\ThreeBody


In [3]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
!pip install ipympl
%matplotlib widget

# Testing matplotlib interactions with a simple plot
fig = plt.figure()
fig.canvas.header_visible = False # Hide the Figure name at the top of the figure
fig.canvas.toolbar_visible = True

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
id = 8462852
# Retrieving all the important details
lc = retrieveKeplerLightCurve(id)
lc

time,flux,flux_err,quality,timecorr,centroid_col,centroid_row,cadenceno,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,sap_quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
,,,,d,pix,pix,,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
object,float32,float32,int32,float32,float64,float64,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
120.53828917000646,1.0000743e+00,5.9970185e-05,0,1.367500e-04,228.97501,133.83864,568,2.3099570e+05,1.4130052e+01,3.1327478e+03,8.4403068e-01,2.6217838e+05,1.5721718e+01,0,nan,nan,nan,nan,228.97501,6.7526351e-05,133.83864,7.8045901e-05,-2.0522515e-03,-4.5541444e-04
120.55872366725089,9.9995077e-01,6.0095972e-05,0,1.376473e-04,228.97466,133.83869,569,2.3098145e+05,1.4129755e+01,3.1327727e+03,8.4375399e-01,2.6214600e+05,1.5754694e+01,0,nan,nan,nan,nan,228.97466,6.7529007e-05,133.83869,7.8055600e-05,-2.3125913e-03,-3.7136092e-04
120.57915806455276,9.9985057e-01,6.0064165e-05,0,1.385446e-04,228.97505,133.83897,570,2.3095938e+05,1.4129274e+01,3.1300862e+03,8.4481192e-01,2.6211973e+05,1.5746356e+01,0,nan,nan,nan,nan,228.97505,6.7525820e-05,133.83897,7.8053832e-05,-1.9923537e-03,-1.7199344e-04
120.59959266192163,1.0000644e+00,6.0178041e-05,0,1.394419e-04,228.97438,133.83980,571,2.3100961e+05,1.4130323e+01,3.1338765e+03,8.4415454e-01,2.6217578e+05,1.5776210e+01,0,nan,nan,nan,nan,228.97438,6.7522087e-05,133.83980,7.8038640e-05,-2.5378668e-03,6.2205142e-04
120.62002715936978,1.0000470e+00,6.0189719e-05,0,1.403394e-04,228.97517,133.83969,572,2.3100167e+05,1.4130178e+01,3.1342488e+03,8.4349722e-01,2.6217122e+05,1.5779271e+01,0,nan,nan,nan,nan,228.97517,6.7518362e-05,133.83969,7.8040648e-05,-2.0599102e-03,7.1742840e-04
120.64046155686083,1.0001690e+00,6.0252100e-05,0,1.412369e-04,228.97498,133.84041,573,2.3102112e+05,1.4130572e+01,3.1325715e+03,8.4332711e-01,2.6220322e+05,1.5795625e+01,0,nan,nan,nan,nan,228.97498,6.7512126e-05,133.84041,7.8033889e-05,-2.1526029e-03,1.3295606e-03
120.66089605443267,1.0000749e+00,6.0253733e-05,0,1.421344e-04,228.97489,133.84050,574,2.3099327e+05,1.4129962e+01,3.1344553e+03,8.4404898e-01,2.6217853e+05,1.5796053e+01,0,nan,nan,nan,nan,228.97489,6.7515219e-05,133.84050,7.8037207e-05,-2.1869289e-03,1.2708127e-03
120.6813306520562,1.0002058e+00,6.0214785e-05,0,1.430321e-04,228.97501,133.84040,575,2.3101480e+05,1.4130332e+01,3.1294685e+03,8.4399694e-01,2.6221284e+05,1.5785842e+01,0,nan,nan,nan,nan,228.97501,6.7511253e-05,133.84040,7.8026918e-05,-2.0245574e-03,1.2980951e-03


In [5]:
plotKeplerLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plotKeplerSAPLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plotKeplerPDCSAPLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
time = np.array(list(map(lambda time: time.value, list(lc.time))))
time

array([ 120.53828917,  120.55872367,  120.57915806, ..., 1587.54721805,
       1587.56765243, 1587.58808682])

In [11]:
flux = np.array(list(map(lambda flux: flux.value, list(lc.pdcsap_flux))))
flux

array([262178.38, 262146.  , 262119.73, ..., 250364.17, 250356.56,
       250382.03], dtype=float32)

In [12]:
evaluateAndProcessData(time, flux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …